### dev

In [2]:
import os
import numpy as np
import pandas as pd
import zarr
import glob
import btrack
import cv2
from PIL import Image, ImageDraw, ImageFont
from tqdm.auto import tqdm

In [8]:
# extract single cell df
# sc_df_fn = '/mnt/SYNO/macrohet_syno/results/dfs/ND4_sc_df.pkl' # time intensive step
acq_ID = (4, 7)
sc_df_fn = f'/mnt/SYNO/macrohet_syno/results/dfs/ND0004/sc_df_{acq_ID[0]}.{acq_ID[1]}.ND0004.pkl'
df = pd.read_pickle(sc_df_fn)


In [3]:
# Prepare font and color settings
font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"  # Update this path to your font file
font = ImageFont.truetype(font_path, 42)
text_color = (0, 255, 255)  # Yellow color in RGB format
# Define the side length
side_length = 750
# and other image properties
track_scale_factor = 5.04
image_scale_m_per_pixel = 1.4949402023919043e-07
image_scale_um_per_pixel = image_scale_m_per_pixel*1E6
image_width = image_height = 6048
last_seg_fn = []

# Bulk generation

In [5]:
df.keys()

Index(['ID', 't', 'x', 'y', 'Mtb Area (µm)', 'Mphi Area (µm)',
       'Infection Status', 'Initial Infection Status',
       'Final Infection Status', 'Unique_ID', 'Strain', 'Compound',
       'Concentration', 'Cell ID', 'Time (hours)', 'Mtb Area Processed (µm)',
       'Time Model (hours)', 'Mtb Area Model (µm)', 'r2', 'Doubling Amounts',
       'Doubling Times'],
      dtype='object')

In [9]:
df

ID    t     x     y  Mtb Area (µm)  \
ID                                                                       
1.4.7.ND0004    0         1.4.7.ND0004    0    70  4350            NaN   
                1         1.4.7.ND0004    1    81  4329     146.225987   
                2         1.4.7.ND0004    2    50  4323     175.413079   
                3         1.4.7.ND0004    3    77  4317     187.861172   
                4         1.4.7.ND0004    4    56  4312            NaN   
...                                ...  ...   ...   ...            ...   
9992.4.7.ND0004 42631  9992.4.7.ND0004  149  2744   523            NaN   
                42632  9992.4.7.ND0004  150  2744   531            NaN   
                42633  9992.4.7.ND0004  151  2748   543      75.582499   
                42634  9992.4.7.ND0004  152  2741   579      73.794622   
                42635  9992.4.7.ND0004  153  2739   535            NaN   

                       Mphi Area (µm) Infection Status  \
ID                                                       
1.4.7.ND0004    0                 NaN              NaN   
                1         1424.178095              NaN   
                2         1445.319740              NaN   
                3         1603.904427              NaN   
                4                 NaN             True   
...                               ...              ...   
9992.4.7.ND0004 42631             NaN              NaN   
                42632             NaN              NaN   
                42633     2539.723929              NaN   
                42634     2473.147860              NaN   
                42635             NaN              NaN   

                       Initial Infection Status  Final Infection Status  \
ID                                                                        
1.4.7.ND0004    0                          True                    True   
                1                          True                    True   
                2                          True                    True   
                3                          True                    True   
                4                          True                    True   
...                                         ...                     ...   
9992.4.7.ND0004 42631                      True                    True   
                42632                      True                    True   
                42633                      True                    True   
                42634                      True                    True   
                42635                      True                    True   

                             Unique_ID  ... Compound Concentration Cell ID  \
ID                                      ...                                  
1.4.7.ND0004    0         1.4.7.ND0004  ...      RIF          EC99       1   
                1         1.4.7.ND0004  ...      RIF          EC99       1   
                2         1.4.7.ND0004  ...      RIF          EC99       1   
                3         1.4.7.ND0004  ...      RIF          EC99       1   
                4         1.4.7.ND0004  ...      RIF          EC99       1   
...                                ...  ...      ...           ...     ...   
9992.4.7.ND0004 42631  9992.4.7.ND0004  ...      RIF          EC99    9992   
                42632  9992.4.7.ND0004  ...      RIF          EC99    9992   
                42633  9992.4.7.ND0004  ...      RIF          EC99    9992   
                42634  9992.4.7.ND0004  ...      RIF          EC99    9992   
                42635  9992.4.7.ND0004  ...      RIF          EC99    9992   

                       Time (hours)  Mtb Area Processed (µm)  \
ID                                                             
1.4.7.ND0004    0               0.0                      NaN   
                1               0.5                      NaN   
                2               1.0                      NaN   
       

In [6]:
N_skipped = 0
N_processed = 0

In [10]:
for unique_ID in tqdm(df['ID'].unique(), total = len(df['ID'].unique())):
    
    try: 
        # if 'ND0003' not in unique_ID:
        #     continue
        if os.path.exists(f"/mnt/SYNO/macrohet_syno/data/glimpses/{unique_ID}.mp4"):
            print(f'Path exists {unique_ID}')
            continue 
            
        sc_df = df[df['ID'] == unique_ID]
        r2 = sc_df['r2'].iloc[0]
        if r2 < 0.7:
            print(f'Skipping ID {unique_ID} as r2 = {r2}')
            N_skipped += 1
            continue

        N_processed +=1
        
        # get the acq id information
        acq_ID = row, column = int(unique_ID.split('.')[1]), int(unique_ID.split('.')[2]) 
        cell_ID = int(unique_ID.split('.')[0])
        expt_ID = unique_ID.split('.')[-1]
        
        # assign the proper channel enumeration
        mphi_channel = 0 if expt_ID == 'PS0000' else 1
        mtb_channel = 1 if expt_ID == 'PS0000' else 0
        
        # if no side length provided then estimate based on max mphi area
        if not side_length:
            # Calculate the side length for cropping based on the square root of the area
            side_length = int(np.sqrt(sc_df['Mphi Area (µm)'].max())) * 2
        
        # preload the images
        image_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/acquisition/zarr/{acq_ID}.zarr'
        zarr_group = zarr.open(image_dir, mode='r')
        images = zarr_group.images
        sample_image = images[0,0,0,...]
        
        
        # Load segmentation if necessary
        seg_fn = glob.glob(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/*/{acq_ID}.h5')[0]
        if seg_fn != last_seg_fn:
            with btrack.io.HDF5FileHandler(seg_fn, 'r', obj_type='obj_type_1') as reader:
                segmentation = reader.segmentation
            last_seg_fn = seg_fn
        
        # load the segmentation 
        rgb_stack = []
        
        # iterate over each frame/data point
        for i, t in tqdm(enumerate(sc_df['Time (hours)']), total = len(sc_df), leave = False, desc = f'Creating glimpse {unique_ID}'):
            
            sc_df_t = sc_df[sc_df['Time (hours)'] == t]
            # Extract xy coordinates and transpose for python and area from the cell information
            y_coord, x_coord, area, t, f, mtb = sc_df_t.loc[:, ['x', 'y', 'Mphi Area (µm)', 'Time (hours)', 'Frame', 'Mtb Area (µm)']].values[0]
        
            # Scale according to tracking shrinkage
            y_coord, x_coord = y_coord * track_scale_factor, x_coord * track_scale_factor
        
            # Calculate the cropping boundaries
            x_start = int(x_coord - side_length / 2)
            x_end = int(x_coord + side_length / 2)
            y_start = int(y_coord - side_length / 2)
            y_end = int(y_coord + side_length / 2)
            
            # Pad the boundaries if they exceed the image dimensions
            if x_start < 0:
                x_pad = abs(x_start)
                x_start = 0
            else:
                x_pad = 0
        
            if x_end > sample_image.shape[0]:
                x_pad_end = x_end - sample_image.shape[0]
                x_end = sample_image.shape[0]
            else:
                x_pad_end = 0
        
            if y_start < 0:
                y_pad = abs(y_start)
                y_start = 0
            else:
                y_pad = 0
        
            if y_end > sample_image.shape[1]:
                y_pad_end = y_end - sample_image.shape[1]
                y_end = sample_image.shape[1]
            else:
                y_pad_end = 0
        
            # Crop the image
            cropped_image = images[int(f), :, 0, x_start:x_end, y_start:y_end]
        
            # Pad the cropped image if necessary
            cropped_image = np.pad(cropped_image, ((0, 0), (x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
            
            # extract the gfp and rfp channels to apply some vis techn
            gfp = cropped_image[mphi_channel, ...]
            rfp = cropped_image[mtb_channel, ...]
            
            # clip the images so that the contrast is more apparent
            contrast_lim_gfp = np.clip(gfp, 358, 5886)
            contrast_lim_rfp = np.clip(rfp, 480, 1300)
            
            norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
            norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
            
            # Create an empty RGB image with the same shape as the input image
            rgb_image = np.zeros((contrast_lim_gfp.shape[0], contrast_lim_gfp.shape[1], 3), dtype=np.uint16)
            
            # Assign the first channel to the green channel of the RGB image
            rgb_image[..., 1] = norm_gfp
            
            # Assign the second channel to the red and blue channels of the RGB image to create magenta
            rgb_image[..., 0] = norm_rfp
            rgb_image[..., 2] = norm_rfp
            
            # scale down to 8bit
            rgb_image = np.uint8(rgb_image >> 8)
        
            
            # load mask (singular)
            cropped_masks = segmentation[int(f), x_start:x_end, y_start:y_end]
            
            # Pad the cropped image if necessary
            cropped_masks = np.pad(cropped_masks, ((x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
        
            # extract only that segment
            seg_ID = cropped_masks[int(cropped_masks.shape[0] / 2), int(cropped_masks.shape[1] / 2)]
           
            if seg_ID == 0:
                instance_mask = np.zeros((side_length, side_length), dtype = np.uint8)
            else:
                instance_mask = (cropped_masks == seg_ID).astype(np.uint8)
        
            # draw outline
            contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(rgb_image, contours, -1, (0, 2 ** 8, 2 ** 8), thickness=2)  # make 8bit
            
            # Convert to PIL image for text overlay
            pil_image = Image.fromarray(rgb_image)
            draw = ImageDraw.Draw(pil_image)
        
            # Bottom left corner text
            bottom_left_text = f"{t} hours"
            draw.text((10, rgb_image.shape[0] - 80), bottom_left_text, font=font, fill=text_color)
        
            # Bottom right corner text
            bottom_right_text = '20µm'
            text_size = font.getbbox(bottom_right_text)
            text_width = text_size[2] - text_size[0]  # Calculate text width
            bottom_right_text_position = (rgb_image.shape[1] - text_width - 40, rgb_image.shape[0] - 80)
            draw.text(bottom_right_text_position, bottom_right_text, font=font, fill=text_color)
        
            # Calculate line length in pixels for a 20 micrometer scale bar
            line_length_pixels = int(20 / image_scale_um_per_pixel)
            line_start = (bottom_right_text_position[0], bottom_right_text_position[1] + 55)
            line_end = (line_start[0] + line_length_pixels, line_start[1])
            draw.line([line_start, line_end], fill=text_color, width=10)
        
            # Top left corner text
            draw.text((10, 10), unique_ID, font=font, fill=text_color)
        
            # Top right corner text
            mtb_value = f"Mtb:{mtb:.2f}µm²"
            text_size = font.getbbox(mtb_value)
            text_width = text_size[2] - text_size[0]  # Calculate text width
            top_right_text_position = (rgb_image.shape[1] - text_width - 10, 10)
            draw.text(top_right_text_position, mtb_value, font=font, fill=text_color)
        
            # Convert back to OpenCV image
            rgb_image = np.array(pil_image)
            
            # Resize image to consistent shape
            rgb_image_resized = cv2.resize(rgb_image, (side_length, side_length), interpolation=cv2.INTER_AREA)

            # add to stack
            rgb_stack.append(rgb_image_resized)
    
        # compile into array
        rgb_stack = np.stack(rgb_stack, axis = 0)
        
        # Get the dimensions of the first frame
        height, width, _ = rgb_stack[0].shape
        
        # Define the frame rate (number of frames per second)
        frame_rate = len(rgb_stack) / 20  # Total frames divided by total seconds
        output_file = f"/mnt/SYNO/macrohet_syno/glimpses/{unique_ID}.mp4"
        # Initialize VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, frame_rate, (width, height))
        
        # Write each frame to the video file
        for frame in rgb_stack:
            out.write(frame)
        
        # Release the VideoWriter object
        out.release()
        
        print(f"Video {unique_ID} saved successfully.")

    except Exception as e:
        error_message = f"Error: {e}\nUnique ID: {unique_ID}\n"
        with open("error_log.txt", "a") as file:
            file.write(error_message)
        

  0%|          | 0/396 [00:00<?, ?it/s]

[INFO][2024/10/28 11:16:35 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(4, 7).h5...


Skipping ID 1.4.7.ND0004 as r2 = -0.18
Skipping ID 10.4.7.ND0004 as r2 = 0.27
Skipping ID 1018.4.7.ND0004 as r2 = 0.04
Skipping ID 10207.4.7.ND0004 as r2 = -1.14


[INFO][2024/10/28 11:16:55 AM] Loading segmentation (154, 6048, 6048)
[INFO][2024/10/28 11:16:55 AM] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(4, 7).h5


Creating glimpse 10224.4.7.ND0004:   0%|          | 0/103 [00:00<?, ?it/s]

...

Creating glimpse 9464.4.7.ND0004:   0%|          | 0/154 [00:00<?, ?it/s]

Creating glimpse 9481.4.7.ND0004:   0%|          | 0/105 [00:00<?, ?it/s]

Skipping ID 9499.4.7.ND0004 as r2 = 0.46
Skipping ID 9549.4.7.ND0004 as r2 = 0.42


Creating glimpse 9711.4.7.ND0004:   0%|          | 0/81 [00:00<?, ?it/s]

Skipping ID 9826.4.7.ND0004 as r2 = 0.16
Skipping ID 9984.4.7.ND0004 as r2 = 0.47
Skipping ID 9992.4.7.ND0004 as r2 = -0.11


In [37]:
df[df['r2'] >= 0.7].drop_duplicates('ID')

,Time (hours),Mtb Area (µm),dMtb Area (µm),Mphi Area (µm),dMphi Area (µm),Infection Status,Initial Infection Status,Final Infection Status,x,y,...,Unique ID,ID,Mtb Area Processed (µm),Time Model (hours),Mtb Area Model (µm),Frame,Edge Status,r2,Doubling Amounts,Doubling Times
23744,0.0,38.349961,36.048069,1323.118349,-624.885348,1.0,1.0,1.0,883.533020,68.502769,...,433.3.3,433.3.3.ND0004,NaN,NaN,NaN,0.0,False,0.95,"[18.7, 37.4]",[39.0]
24092,0.0,17.744679,51.356766,693.115203,-261.856930,1.0,1.0,1.0,889.058899,115.861832,...,505.3.3,505.3.3.ND0004,NaN,NaN,NaN,0.0,False,0.82,"[15.9, 31.8]",[27.5]
24211,0.0,83.091582,66.062054,414.899199,-224.266817,1.0,1.0,1.0,245.561722,306.898987,...,424.3.3,424.3.3.ND0004,NaN,NaN,NaN,0.0,False,0.87,None,None
24298,0.0,0.000000,58.664713,436.063192,404.037846,0.0,0.0,1.0,298.399902,166.023849,...,364.3.3,364.3.3.ND0004,NaN,NaN,NaN,0.0,False,0.88,None,None
24598,0.0,18.169300,84.231354,634.808066,-53.435173,1.0,1.0,1.0,369.461548,190.425751,...,375.3.3,375.3.3.ND0004,NaN,NaN,NaN,0.0,False,0.88,"[18.7, 37.4]",[41.5]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801374,27.5,13.118547,-8.380673,1299.808903,56.921533,1.0,1.0,1.0,334.461151,1079.291992,...,3728.6.12,3728.6.12.ND0004,NaN,NaN,NaN,55.0,False,0.76,"[5.0, 10.0]",[16.5]
801473,28.0,0.000000,44.853363,978.728549,1634.298336,0.0,0.0,1.0,974.366882,364.910248,...,3804.6.12,3804.6.12.ND0004,NaN,NaN,NaN,56.0,False,0.88,"[1.92, 3.84, 7.68, 15.36, 30.72]","[1.5, 4.5, 13.5, 6.0]"
801571,30.0,3.687496,-0.938635,533.994153,295.603108,1.0,1.0,1.0,602.169861,502.689240,...,3984.6.12,3984.6.12.ND0004,NaN,NaN,NaN,60.0,False,0.79,None,None
802011,37.0,0.000000,27.868532,208.645242,1635.728637,0.0,0.0,1.0,385.296692,1096.364868,...,4585.6.12,4585.6.12.ND0004,NaN,NaN,NaN,74.0,False,0.82,"[1.92, 3.84, 7.68, 15.36]","[0.5, 1.5, 2.5]"


# Single instance

In [5]:
df['ID'].unique()

array(['282.3.1.ND0004', '216.3.1.ND0004', '241.3.1.ND0004', ...,
       '4912.6.12.ND0004', '4954.6.12.ND0004', '5015.6.12.ND0004'],
      dtype=object)

In [7]:
unique_ID = '282.3.1.ND0004' # '211.3.5.PS0000'

In [12]:
df

,Time (hours),Mtb Area (µm),dMtb Area (µm),Mphi Area (µm),dMphi Area (µm),Infection Status,Initial Infection Status,Final Infection Status,x,y,...,Concentration,Cell ID,Acquisition ID,Experiment ID,Unique ID,ID,Mtb Area Processed (µm),Time Model (hours),Mtb Area Model (µm),Frame
0,0.0,0.000000,0.000000,1430.122786,-684.600439,0.0,0.0,0.0,535.972046,36.392685,...,EC0,282,"(3, 1)",ND0004,282.3.1,282.3.1.ND0004,NaN,NaN,NaN,0.0
1,0.5,0.000000,0.000000,1553.352206,-684.600439,0.0,0.0,0.0,517.281433,31.998940,...,EC0,282,"(3, 1)",ND0004,282.3.1,282.3.1.ND0004,NaN,NaN,NaN,1.0
2,1.0,0.000000,0.000000,2206.284874,-684.600439,0.0,0.0,0.0,503.972534,27.664877,...,EC0,282,"(3, 1)",ND0004,282.3.1,282.3.1.ND0004,NaN,NaN,NaN,2.0
3,1.5,NaN,0.000000,NaN,-684.600439,NaN,0.0,0.0,493.696564,22.767044,...,EC0,282,"(3, 1)",ND0004,282.3.1,282.3.1.ND0004,NaN,NaN,NaN,3.0
4,2.0,0.000000,0.000000,806.824178,-684.600439,0.0,0.0,0.0,477.338898,15.456922,...,EC0,282,"(3, 1)",ND0004,282.3.1,282.3.1.ND0004,NaN,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802672,74.5,95.562024,-38.908672,945.518734,-1344.930448,1.0,1.0,1.0,432.996796,159.980438,...,EC99,5015,"(6, 12)",ND0004,5015.6.12,5015.6.12.ND0004,85.88514,NaN,NaN,149.0
802673,75.0,94.623388,-38.908672,1361.825886,-1344.930448,1.0,1.0,1.0,433.311157,166.972198,...,EC99,5015,"(6, 12)",ND0004,5015.6.12,5015.6.12.ND0004,85.88514,NaN,NaN,150.0
802674,75.5,83.091582,-38.908672,1152.510190,-1344.930448,1.0,1.0,1.0,435.046844,160.506119,...,EC99,5015,"(6, 12)",ND0004,5015.6.12,5015.6.12.ND0004,85.661655,NaN,NaN,151.0
802675,76.0,77.750300,-38.908672,1126.139005,-1344.930448,1.0,1.0,1.0,433.829651,157.600281,...,EC99,5015,"(6, 12)",ND0004,5015.6.12,5015.6.12.ND0004,84.331922,NaN,NaN,152.0


In [14]:
sc_df = df[df['ID'] == unique_ID]
        
# get the acq id information
acq_ID = row, column = int(unique_ID.split('.')[1]), int(unique_ID.split('.')[2]) 
cell_ID = int(unique_ID.split('.')[0])
expt_ID = unique_ID.split('.')[-1]

# assign the proper channel enumeration
mphi_channel = 0 if expt_ID == 'PS0000' else 1
mtb_channel = 1 if expt_ID == 'PS0000' else 0

# if no side length provided then estimate based on max mphi area
if not side_length:
    # Calculate the side length for cropping based on the square root of the area
    side_length = int(np.sqrt(sc_df['Mphi Area (µm)'].max())) * 2

# preload the images
image_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/acquisition/zarr/{acq_ID}.zarr'
zarr_group = zarr.open(image_dir, mode='r')
images = zarr_group.images
sample_image = images[0,0,0,...]


# Load segmentation if necessary
seg_fn = glob.glob(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/*/{acq_ID}.h5')[0]
if seg_fn != last_seg_fn:
    with btrack.io.HDF5FileHandler(seg_fn, 'r', obj_type='obj_type_1') as reader:
        segmentation = reader.segmentation
    last_seg_fn = seg_fn

# load the segmentation 
rgb_stack = []

# iterate over each frame/data point
for i, t in tqdm(enumerate(sc_df['Time (hours)']), total = len(sc_df), leave = False, desc = f'Creating glimpse {unique_ID}'):
    
    sc_df_t = sc_df[sc_df['Time (hours)'] == t]
    # Extract xy coordinates and transpose for python and area from the cell information
    y_coord, x_coord, area, t, f, mtb = sc_df_t.loc[:, ['x', 'y', 'Mphi Area (µm)', 'Time (hours)', 'Frame', 'Mtb Area (µm)']].values[0]

    # Scale according to tracking shrinkage
    y_coord, x_coord = y_coord * track_scale_factor, x_coord * track_scale_factor

    # Calculate the cropping boundaries
    x_start = int(x_coord - side_length / 2)
    x_end = int(x_coord + side_length / 2)
    y_start = int(y_coord - side_length / 2)
    y_end = int(y_coord + side_length / 2)
    
    # Pad the boundaries if they exceed the image dimensions
    if x_start < 0:
        x_pad = abs(x_start)
        x_start = 0
    else:
        x_pad = 0

    if x_end > sample_image.shape[0]:
        x_pad_end = x_end - sample_image.shape[0]
        x_end = sample_image.shape[0]
    else:
        x_pad_end = 0

    if y_start < 0:
        y_pad = abs(y_start)
        y_start = 0
    else:
        y_pad = 0

    if y_end > sample_image.shape[1]:
        y_pad_end = y_end - sample_image.shape[1]
        y_end = sample_image.shape[1]
    else:
        y_pad_end = 0

    # Crop the image
    cropped_image = images[int(f), :, 0, x_start:x_end, y_start:y_end]

    # Pad the cropped image if necessary
    cropped_image = np.pad(cropped_image, ((0, 0), (x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
    
    # extract the gfp and rfp channels to apply some vis techn
    gfp = cropped_image[mphi_channel, ...]
    rfp = cropped_image[mtb_channel, ...]
    
    # clip the images so that the contrast is more apparent
    contrast_lim_gfp = np.clip(gfp, 358, 5886)
    contrast_lim_rfp = np.clip(rfp, 480, 1300)
    
    norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    
    # Create an empty RGB image with the same shape as the input image
    rgb_image = np.zeros((contrast_lim_gfp.shape[0], contrast_lim_gfp.shape[1], 3), dtype=np.uint16)
    
    # Assign the first channel to the green channel of the RGB image
    rgb_image[..., 1] = norm_gfp
    
    # Assign the second channel to the red and blue channels of the RGB image to create magenta
    rgb_image[..., 0] = norm_rfp
    rgb_image[..., 2] = norm_rfp
    
    # scale down to 8bit
    rgb_image = np.uint8(rgb_image >> 8)

    
    # load mask (singular)
    cropped_masks = segmentation[int(f), x_start:x_end, y_start:y_end]
    
    # Pad the cropped image if necessary
    cropped_masks = np.pad(cropped_masks, ((x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')

    # extract only that segment
    seg_ID = cropped_masks[int(cropped_masks.shape[0] / 2), int(cropped_masks.shape[1] / 2)]
   
    if seg_ID == 0:
        instance_mask = np.zeros((side_length, side_length), dtype = np.uint8)
    else:
        instance_mask = (cropped_masks == seg_ID).astype(np.uint8)

    # draw outline
    contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(rgb_image, contours, -1, (0, 2 ** 8, 2 ** 8), thickness=2)  # make 8bit
    
    # Convert to PIL image for text overlay
    pil_image = Image.fromarray(rgb_image)
    draw = ImageDraw.Draw(pil_image)

    # Bottom left corner text
    bottom_left_text = f"{t} hours"
    draw.text((10, rgb_image.shape[0] - 80), bottom_left_text, font=font, fill=text_color)

    # Bottom right corner text
    bottom_right_text = '20µm'
    text_size = font.getbbox(bottom_right_text)
    text_width = text_size[2] - text_size[0]  # Calculate text width
    bottom_right_text_position = (rgb_image.shape[1] - text_width - 40, rgb_image.shape[0] - 80)
    draw.text(bottom_right_text_position, bottom_right_text, font=font, fill=text_color)

    # Calculate line length in pixels for a 20 micrometer scale bar
    line_length_pixels = int(20 / image_scale_um_per_pixel)
    line_start = (bottom_right_text_position[0], bottom_right_text_position[1] + 55)
    line_end = (line_start[0] + line_length_pixels, line_start[1])
    draw.line([line_start, line_end], fill=text_color, width=10)

    # Top left corner text
    draw.text((10, 10), unique_ID, font=font, fill=text_color)

    # Top right corner text
    mtb_value = f"Mtb:{mtb:.2f}µm²"
    text_size = font.getbbox(mtb_value)
    text_width = text_size[2] - text_size[0]  # Calculate text width
    top_right_text_position = (rgb_image.shape[1] - text_width - 10, 10)
    draw.text(top_right_text_position, mtb_value, font=font, fill=text_color)

    # Convert back to OpenCV image
    rgb_image = np.array(pil_image)
    
    # Resize image to consistent shape
    rgb_image_resized = cv2.resize(rgb_image, (side_length, side_length), interpolation=cv2.INTER_AREA)

    # add to stack
    rgb_stack.append(rgb_image_resized)

# compile into array
rgb_stack = np.stack(rgb_stack, axis = 0)

# Get the dimensions of the first frame
height, width, _ = rgb_stack[0].shape

# Define the frame rate (number of frames per second)
frame_rate = len(rgb_stack) / 20  # Total frames divided by total seconds
output_file = f"/mnt/SYNO/macrohet_syno/glimpses/{unique_ID}.mp4"
# Initialize VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, frame_rate, (width, height))

# Write each frame to the video file
for frame in rgb_stack:
    out.write(frame)

# Release the VideoWriter object
out.release()

print(f"Video {unique_ID} saved successfully.")

Creating glimpse 282.3.1.ND0004:   0%|          | 0/121 [00:00<?, ?it/s]

Video 282.3.1.ND0004 saved successfully.



(python:1361987): GStreamer-CRITICAL **: 14:21:41.351: gst_element_make_from_uri: assertion 'gst_uri_is_valid (uri)' failed
[ WARN:0@429.695] global /croot/opencv-suite_1691620365762/work/modules/videoio/src/cap_gstreamer.cpp (2180) open OpenCV | GStreamer warning: cannot link elements
